In [ ]:
!pip install --upgrade pip
!pip install --upgrade "datasets[audio]" transformers accelerate evaluate jiwer tensorboard gradio


In [ ]:
# generate a token from your hf account n use it so the model can be saved to the hub
from huggingface_hub import notebook_login

notebook_login()

Run this only in colab.

In [ ]:
# download the "dataset" folder from onedrive and put this "dataset" folder in your google drive once, then use it because downloading from hf takes time on colab only
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive

In [ ]:
from datasets import load_dataset, DatasetDict

my_dataset = DatasetDict()
my_dataset["train"] = load_dataset("/content/drive/My Drive/dataset", split='train') # USE THIS ON COLAB
print(my_dataset)

Run this on your machine.

In [ ]:
from datasets import load_dataset, DatasetDict

my_dataset = DatasetDict()
my_dataset["train"] = load_dataset("itskavya/gp", split='train') # USE THIS ON YOUR MACHINE
print(my_dataset)


Preprocessing

In [ ]:
# Get column names to identify the second column
column_names = my_dataset["train"].column_names
second_column = column_names[1]  # Get the second column name
second_column

In [ ]:
# filter out rows containing '[موسيقى]'

def filter_dataset(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if '[موسيقى]' not in dataset['transcription'][i]:
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

filtered_dataset = filter_dataset(my_dataset['train'])

print(filtered_dataset)

my_dataset['train'] = filtered_dataset

print(my_dataset)


In [6]:
import re

# filter rows that have anything other than letters
def filter_dataset(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if bool(re.search(r'[^A-Za-z\u0600-\u06FF\s]', dataset['transcription'][i])):
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

filtered_dataset2 = filter_dataset(my_dataset['train'])

print(filtered_dataset2)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 23
})


In [7]:
for i in range(len(filtered_dataset2)):
  print(filtered_dataset2[i]['audio']['path'])
  print(filtered_dataset2[i]['transcription'])
  print("\n")

/content/drive/My Drive/dataset/train/audio/10_154.mp3
السلامه ‏m


/content/drive/My Drive/dataset/train/audio/12_chunk_10.wav
حلو لعندك شخص قوي عزومك وإرادتك لكن كيف يكون هذا الشخص موجود وانت أصلا ما عندك لا عزوم ولا إرادة أيضا عدم القدرة على التكيف مع التغيير وهيمنة التفكير العاطفي عليك كثرة التفكير بالمواقف السلبية بالماضي ممكن تلقى أحد هالعلمات أو الأعراض يمثلك لكن ما يعني انك هش جدا لكن ليش وصلت ل...


/content/drive/My Drive/dataset/train/audio/1_245_0.mp3
9 بالليل راح يكون في بث ويا اخوان راح


/content/drive/My Drive/dataset/train/audio/2_162_0.mp3
4اربعه على اربعه يمكن في يمكن اربعميه  اربع الاف ما


/content/drive/My Drive/dataset/train/audio/3_108_0.mp3
اقسم بالله انك اسطوره اللي يتابعني من صفر مشترك لين 14


/content/drive/My Drive/dataset/train/audio/3_124_0.mp3
مثل ري ردمشن تو او جراند 5 فايف او كول اوف


/content/drive/My Drive/dataset/train/audio/5_186.mp3
‏ok


/content/drive/My Drive/dataset/train/audio/6_210.mp3
يحفظكم ‏


/content/drive/My Drive/dataset/train/audio

In [8]:
# filter rows that have both eng and arabic
def filter_dataset(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if bool(re.search(r'[A-Za-z]', dataset['transcription'][i]) and re.search(r'[\u0600-\u06FF]', dataset['transcription'][i])):
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

filtered_dataset3 = filter_dataset(my_dataset['train'])

print(filtered_dataset3)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 10
})


In [9]:
for i in range(len(filtered_dataset3)):
  print(filtered_dataset3[i]['audio']['path'])
  print(filtered_dataset3[i]['transcription'])
  print("\n")

/content/drive/My Drive/dataset/train/audio/10_154.mp3
السلامه ‏m


/content/drive/My Drive/dataset/train/audio/11_chunk_6.wav
متأكد ثلاث ايام لاتكلم و عطاني أربع علاجات مضاد و شراب مادري ايش   عاطاني كذا كم حاجه روح روح و لم قبل أن تبدأ المقطع أنت استراتيجي أنت عبقري أنت تحب البحار عادل جايب لك اليوم لعبة World of Warships أو عالم السفن الحربية اللعبة هذه يااخوان موجودة على البي سي و مجانية اللعبة عبارة عن معارك بحرية و السفن موجودة أكثر من خمسمئة سفينة موزعة على أحد عشر دولة و السفن هذا


/content/drive/My Drive/dataset/train/audio/9_chunk_10.wav
الفيروس هذا اللي حملته اسمه no escape يعني ما فيه هرب سوي لها extraction ويشوفي شي صير تحذير للمرة الثانية سوي لها تشغيل على أي حال على طول ما عطانا حتى مقدمات ولا الأصوات المزينية ما فيه على طول حذر سوانا عد التشغيل ما كنا حذر


/content/drive/My Drive/dataset/train/audio/9_chunk_23.wav
طيب فتح لي task manager خلوني أحاول أرجع للoperating system بدون ما أعطيه فلوس خلوني أشوف anti-malware ما منه فايدة شغال كده بس دهني قفلته ما نبغى نظلمها مر

Checking whether the tokenizer can handle the english text

In [10]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="Arabic", task="transcribe")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [18]:
input_str = filtered_dataset3[2]["transcription"]
labels = tokenizer(input_str).input_ids # it returns a dict of input ids and attention mask so just get the input ids
print(labels)
print(tokenizer.tokenize(input_str))
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

[50258, 50272, 50359, 50363, 6027, 5172, 13546, 41779, 23758, 13672, 1829, 11331, 42213, 47395, 24525, 2304, 3224, 572, 7615, 37495, 22653, 19446, 8978, 3224, 8032, 25513, 8608, 45865, 5296, 11296, 30197, 4032, 1829, 8592, 38688, 1829, 44049, 20328, 13546, 6055, 5016, 8848, 13546, 5296, 19528, 25720, 6024, 104, 7649, 10632, 8608, 45865, 5296, 11296, 6055, 8592, 17082, 26895, 15844, 36632, 11331, 6027, 15844, 23032, 12610, 19446, 6225, 9566, 7649, 995, 11331, 49975, 3714, 28543, 2304, 9307, 49429, 16247, 9381, 2407, 9307, 9673, 11622, 9957, 10632, 19446, 8978, 3224, 15844, 23032, 12610, 11331, 8848, 2288, 8608, 2407, 7649, 995, 6225, 3215, 16712, 8592, 17082, 26895, 19446, 9122, 8315, 11331, 8848, 2288, 50257]
['Ø§ÙĦ', 'Ùģ', 'ÙĬØ±', 'ÙĪØ³', 'ĠÙĩØ°Ø§', 'ĠØ§ÙĦÙĦ', 'ÙĬ', 'ĠØŃ', 'ÙħÙĦ', 'ØªÙĩ', 'ĠØ§Ø³', 'Ùħ', 'Ùĩ', 'Ġno', 'Ġescape', 'ĠÙĬØ¹', 'ÙĨÙĬ', 'ĠÙħØ§', 'ĠÙģÙĬ', 'Ùĩ', 'ĠÙĩ', 'Ø±Ø¨', 'ĠØ³', 'ÙĪÙĬ', 'ĠÙĦ', 'ÙĩØ§', 'Ġextraction', 'ĠÙĪ', 'ÙĬ', 'Ø´', 'ÙĪÙģ', 'ÙĬ', 'ĠØ´ÙĬ', 'ĠØµ', 'ÙĬØ±', 'Ġ

In [12]:
!pip install PyArabic

In [38]:
import pyarabic.araby as araby

before_filter="هو أننا فقط أخذنا وقت طويل جداً بالتفكير بالماضي حيناً وبالمستقبل حيناً آخر في نفس الفترة الزمنية وراح الحاضر بدون ما نحس هذا بالضبط اللي خلينا نفقد التركيز ويسمح للقلق والتوتر بالسيطرة على عقولنا وأفكارنا وهنا تجي فائدة اللي يقظه الذهنية اللي تساعدنا على تجنب هذا التأثير السلبي"
after_filter = araby.strip_diacritics(before_filter)
print(before_filter)
print(after_filter)
print(before_filter==after_filter)


هو أننا فقط أخذنا وقت طويل جداً بالتفكير بالماضي حيناً وبالمستقبل حيناً آخر في نفس الفترة الزمنية وراح الحاضر بدون ما نحس هذا بالضبط اللي خلينا نفقد التركيز ويسمح للقلق والتوتر بالسيطرة على عقولنا وأفكارنا وهنا تجي فائدة اللي يقظه الذهنية اللي تساعدنا على تجنب هذا التأثير السلبي
هو أننا فقط أخذنا وقت طويل جدا بالتفكير بالماضي حينا وبالمستقبل حينا آخر في نفس الفترة الزمنية وراح الحاضر بدون ما نحس هذا بالضبط اللي خلينا نفقد التركيز ويسمح للقلق والتوتر بالسيطرة على عقولنا وأفكارنا وهنا تجي فائدة اللي يقظه الذهنية اللي تساعدنا على تجنب هذا التأثير السلبي
False


So only clean the text as in remove the punctuation and diacritics, the other records are deleted/fixed from the dataset itself.

In [ ]:
# remove full stop and question mark
def remove_fullstop_and_questionmark(text):
    return re.sub(r'[.?]', '', text)

def clean_text(example):
    example['transcription'] = remove_fullstop_and_questionmark(example['transcription'])
    return example

my_dataset['train'] = my_dataset['train'].map(clean_text)

In [ ]:
# check if removed
filtered_dataset2 = filter_dataset(my_dataset['train'])

print(filtered_dataset2)

for i in range(len(filtered_dataset2)):
  print(filtered_dataset2[i]['audio']['path'])
  print(filtered_dataset2[i]['transcription'])
  print("\n")


Dataset({
    features: ['audio', 'transcription'],
    num_rows: 9
})
/content/drive/My Drive/dataset/train/audio/10_154.mp3
السلامه ‏m


/content/drive/My Drive/dataset/train/audio/1_245_0.mp3
9 بالليل راح يكون في بث ويا اخوان راح


/content/drive/My Drive/dataset/train/audio/2_162_0.mp3
4اربعه على اربعه يمكن في يمكن اربعميه  اربع الاف ما


/content/drive/My Drive/dataset/train/audio/3_108_0.mp3
اقسم بالله انك اسطوره اللي يتابعني من صفر مشترك لين 14


/content/drive/My Drive/dataset/train/audio/3_124_0.mp3
مثل ري ردمشن تو او جراند 5 فايف او كول اوف


/content/drive/My Drive/dataset/train/audio/5_186.mp3
‏ok


/content/drive/My Drive/dataset/train/audio/6_210.mp3
يحفظكم ‏


/content/drive/My Drive/dataset/train/audio/9_chunk_23.wav
طيب فتح لي task manager خلوني أحاول أرجع للoperating system بدون ما أعطيه فلوس خلوني أشوف anti-malware ما منه فايدة شغال كده بس دهني قفلته ما نبغى نظلمها مرة طيب أنا ما حصلت الرانسموير خلوني أحاول أطفل ويندوز وأشغلها مرة ثانية ونشوف اشي يصير طيب سوينا عادة 

In [36]:
# remove diacritic
def remove_diacritic(example):
    example['transcription'] = araby.strip_diacritics(example['transcription'])
    return example

my_dataset['train'] = my_dataset['train'].map(remove_diacritic)
print(my_dataset)

Map:   0%|          | 0/4916 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 4916
    })
})


Hear some audios

In [ ]:
import random

rand_int = random.randint(0, len(my_dataset['train'])-1)
print(my_dataset['train'][rand_int])

{'audio': {'path': '/content/drive/My Drive/dataset/train/audio/7_219.mp3', 'array': array([-0.20119352, -0.2439578 , -0.18837818, ...,  0.00101169,
        0.0014196 ,  0.00118142]), 'sampling_rate': 44100}, 'transcription': 'الاضاءه اللي قدامي وهذا بالضبط اللي سار'}


In [ ]:
import IPython.display as ipd

print(my_dataset['train'][rand_int]["transcription"])
ipd.Audio(data=my_dataset['train'][rand_int]["audio"]["array"], autoplay=True, rate=my_dataset['train'][rand_int]["audio"]["sampling_rate"])


الاضاءه اللي قدامي وهذا بالضبط اللي سار


In [ ]:
# split the dataset for testing
split_dataset = my_dataset["train"].train_test_split(test_size=0.2)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 3932
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 984
    })
})

In [ ]:
my_dataset['train'] = split_dataset['train']
my_dataset['test'] = split_dataset['test']
my_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 3932
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 984
    })
})

In [ ]:
# input to whisper should be log-mel, this is done automatically by the whisper feature extractor
# it also performs padding and truncation
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

# load the whisper tokenizer to convert map the indices predicted by model to text
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="Arabic", task="transcribe")


In [ ]:
input_str = my_dataset["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids # it returns a dict of input ids and attention mask so just get the input ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 اكيد مصدر رزق ولا انا ما كملت اصلا معه
Decoded w/ special:    <|startoftranscript|><|ar|><|transcribe|><|notimestamps|>اكيد مصدر رزق ولا انا ما كملت اصلا معه<|endoftext|>
Decoded w/out special: اكيد مصدر رزق ولا انا ما كملت اصلا معه
Are equal:             True


In [ ]:
# can combine the tokenizer and feature extractor into one object
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="Arabic", task="transcribe")


In [ ]:
print(my_dataset["train"][0])

{'audio': {'path': '/content/drive/My Drive/dataset/train/audio/1_101_0.mp3', 'array': array([ 8.97481591e-02,  1.05253085e-01,  7.79567659e-02, ...,
       -4.64100740e-05,  7.49935280e-05,  4.08802880e-05]), 'sampling_rate': 44100}, 'transcription': 'اكيد مصدر رزق ولا انا ما كملت اصلا معه'}


In [ ]:
# need to sample the audio to match whisper's sampling rate, this does it on the fly when audio is loaded
from datasets import Audio

my_dataset = my_dataset.cast_column("audio", Audio(sampling_rate=16000))


In [ ]:

print(my_dataset["train"][0])

{'audio': {'path': '/content/drive/My Drive/dataset/train/audio/1_101_0.mp3', 'array': array([ 0.062504  ,  0.08967225,  0.04713362, ...,  0.00048709,
       -0.00020194,  0.00028571]), 'sampling_rate': 16000}, 'transcription': 'اكيد مصدر رزق ولا انا ما كملت اصلا معه'}


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0] # its a batch

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch


In [ ]:
my_dataset = my_dataset.map(prepare_dataset, remove_columns=my_dataset.column_names["train"], num_proc=4) # use num_proc=4 to make it process faster, if gives error remove it


Map (num_proc=4):   0%|          | 0/3932 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/984 [00:00<?, ? examples/s]

In [ ]:
# load the model
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [ ]:
# tokens in transcript can't be more than 448
max_label_length = model.config.max_length
def is_labels_in_length_range(labels):
    return len(labels) < max_label_length

my_dataset = my_dataset.filter(is_labels_in_length_range, num_proc=4, input_columns=["labels"])
print(my_dataset)

In [ ]:
model.generation_config.language = "Arabic"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None # don't use the legacy method instead use the config above


In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass #decorator that provides init function
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt") # pad the input audio and return tensors

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt") # pad the transcript and return tensors

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100) #ne not equal to 1, means get padding tokens from attention mask and replace with -100 so the loss function can ignore them

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)


In [ ]:
import evaluate

metric = evaluate.load("wer")


In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True) # use batch decode to get literal tokens for calculating error
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-medium-informal-arabic",  # change small if diff checkpoint
    per_device_train_batch_size=16, # this can be reduced if out of memory
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size, accumulate gradients before updating weights when using big batch size to help w memory
    learning_rate=1e-5,
    warmup_steps=500, # for lr
    max_steps=3000, # train for max 5000 steps
    gradient_checkpointing=True, # keep subset of activatons in fp n calculate again in bp for memory
    fp16=True, # mixed preciison training with 16 bits instead of 32 for faster training n memory
    evaluation_strategy="steps", # steps not epoch
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225, # tokens
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False, # because lower wer is better
    push_to_hub=True,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=my_dataset["train"],
    eval_dataset=my_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


<ipython-input-25-6433bd8ae229>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.051500,0.461925,30.449763
2000,0.002800,0.529558,28.289412
3000,0.000500,0.548619,27.516529


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 2

TrainOutput(global_step=3000, training_loss=0.10574304424319417, metrics={'train_runtime': 18503.0069, 'train_samples_per_second': 2.594, 'train_steps_per_second': 0.162, 'total_flos': 4.894004652539904e+19, 'train_loss': 0.10574304424319417, 'epoch': 12.195121951219512})

In [ ]:
# write meta data
kwargs = {
    "dataset_tags": "itskavya/gp",
    "dataset": "Informal Arabic",
    "language": ["ar"],
    "model_name": "Whisper Medium Informal Arabic",
    "finetuned_from": "openai/whisper-medium", # should be changed if we use diff checkpoint
    "tags": ["automatic-speech-recognition", "arabic"],
    "tasks": "automatic-speech-recognition",
}
trainer.push_to_hub(**kwargs)


CommitInfo(commit_url='https://huggingface.co/itskavya/whisper-medium-informal-arabic/commit/3c87e8e629655bfdd85079d48ef22f7e858ea8ad', commit_message='End of training', commit_description='', oid='3c87e8e629655bfdd85079d48ef22f7e858ea8ad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/itskavya/whisper-medium-informal-arabic', endpoint='https://huggingface.co', repo_type='model', repo_id='itskavya/whisper-medium-informal-arabic'), pr_revision=None, pr_num=None)

References:

-https://huggingface.co/blog/fine-tune-whisper
